<a href="https://colab.research.google.com/github/yonishat/my-repository/blob/main/VAE_Based_Real_Time_Anomaly_Detection_Approach_for_En_hanced_V2X_Communication_Security.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchinfo

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision
import torchvision.datasets as datasets
from torchinfo import summary


import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print('PyTorch version:', torch.__version__, '\n')
print('GPU name:', torch.cuda.get_device_name(), '\n')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device is:', device, '\n')
print('Total number of GPUs:', torch.cuda.device_count())

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

# Load and preprocess the dataset
file_path = '/content/modified_filtered_data.csv'
data = pd.read_csv(file_path)

anomaly_data = data[data['time']>=900].reset_index(drop=True)

dataset = data[data['time'] < 900].reset_index(drop=True)

# Split the remaining data into training (90%) and testing (10%)
train_data = dataset[dataset['time'] < 800].reset_index(drop=True)
test_data = dataset[dataset['time'] >= 800].reset_index(drop=True)

anomaly_data1 = anomaly_data.drop(columns=[ 'angle','id', 'time')

# Drop unwanted columns for training data
train_data1 = train_data.drop(columns=[ 'angle','id', 'time')

# Normalize other columns using MinMaxScaler

min_max_scaler = MinMaxScaler(feature_range=(0, 1))

data_normalized_train = min_max_scaler.fit_transform(train_data1)
data_normalized_train = pd.DataFrame(data_normalized_train, columns=train_data1.columns)
# Process the test data similarly
test_data1 = test_data.drop(columns=['angle', 'id', 'time')

data_normalized_test = min_max_scaler.transform(test_data1)
data_normalized_test = pd.DataFrame(data_normalized_test, columns=test_data1.columns)

# Process the anoamly data similarly
data_normalized_anomaly = min_max_scaler.transform(anomaly_data1)
data_normalized_anomaly = pd.DataFrame(data_normalized_anomaly, columns=anomaly_data1.columns)

train_data1.shape, anomaly_data1.shape, test_data1.shape

**Select the anomaly to inject from the following options**


1.   Option 1 shows for constatnt speed offset now, but can be changed to position offset by changing a few details.
2.   Option 2 shows for Targeted vehicle speed offset now, but can be changed to position offset by changing a few details.

---



In [ ]:
# Optinon 1
# Define function to inject constant offset anomalies
def inject_constant_offset(anomaly_data, anomaly_percentage=0.3, high_value_factor= 0.6):
    """Inject constant offset anomalies into the middle 30% of the test dataset."""
    total_rows = len(anomaly_data)
    num_anomalies = int(total_rows * anomaly_percentage)
    if 'speed' in anomaly_data.columns:
        max_speed = anomaly_data['speed'].max()
        #max_x = anomaly_data['x'].max()
        #max_y = anomaly_data['y'].max()

        high_value = max_speed * high_value_factor
        #high_value_x = max_x * high_value_factor
        #high_value_y = max_y * high_value_factor

    else:
        raise ValueError("Dataset does not contain a 'speed' column.")
    # Determine the start and end indices for the middle 20% of the dataset
    start_index = max(0, (total_rows // 2) - (num_anomalies // 2))  # Ensure start_index is not negative
    end_index = min(start_index + num_anomalies, total_rows)  # Ensure end_index is within bounds

    # Make sure indices are valid
    if start_index >= total_rows:
        raise ValueError("The calculated start index is out of bounds of the dataset.")
    if end_index > total_rows:
        end_index = total_rows

    anomaly_indices = np.arange(start_index, end_index)

    # Create a copy of the data to modify
    data_with_anomalies = anomaly_data.copy()

    # Inject constant speed offset anomaly at the selected indices
    if 'speed' in data_with_anomalies.columns:
        for index in anomaly_indices:

            data_with_anomalies.iloc[index, data_with_anomalies.columns.get_loc('speed')] += high_value
            #data_with_anomalies.iloc[index, data_with_anomalies.columns.get_loc('x')] += high_value_x
            #data_with_anomalies.iloc[index, data_with_anomalies.columns.get_loc('y')] += high_value_y


    else:
        raise ValueError("Dataset does not contain a 'speed' column.")

    return data_with_anomalies, anomaly_indices

# Inject anomalies into the middle 20% of the test dataset
anomalous_test_data, test_anomaly_indices_offset = inject_constant_speed_offset(anomaly_data, anomaly_percentage=0.3, high_value_factor=0.6)

# Generate anomaly labels for the test data
anomaly_labels_test = np.zeros(len(anomaly_data), dtype=int)

# Ensure anomaly indices are within the range of test_data length
test_anomaly_indices_offset = [i for i in test_anomaly_indices_offset if i < len(anomaly_data)]
anomaly_labels_test[test_anomaly_indices_offset] = 1

# Display the results
print(f"Labels (0 for normal, 1 for anomalous): {anomaly_labels_test}")
anomaly_labels_test.shape


In [ ]:
# Option 2
# Define function to inject_Targeted_vehicle_offset anomalies for a specific vehicle
def inject_Targeted_vehicle_offset(anomaly_data, vehicle_id, high_value_factor=0.4):

    # Check if required columns exist
    if 'speed' not in anomaly_data.columns or 'id' not in anomaly_data.columns:
        raise ValueError("Dataset must contain 'speed' and 'id' columns.")

    # Filter the dataset for the selected vehicle
    vehicle_data = anomaly_data[anomaly_data['id'] == vehicle_id]
    if vehicle_data.empty:
        raise ValueError(f"No data found for vehicle ID {vehicle_id}.")

    # Calculate the anomaly magnitude
    max_speed = anomaly_data['speed'].max()
    #max_x = anomaly_data['x'].max()
    #max_y = anomaly_data['y'].max()

    high_value = max_speed * high_value_factor
    #high_value_x = max_x * high_value_factor
    #high_value_y = max_y * high_value_factor

    # Get the indices of the selected vehicle's rows
    anomaly_indices = vehicle_data.index

    # Create a copy of the data to modify
    data_with_anomalies = anomaly_data.copy()

    # Inject anomalies at the selected indices
    data_with_anomalies.loc[anomaly_indices, 'speed'] += high_value
    #data_with_anomalies.loc[anomaly_indices, 'x'] += high_value_x
    #data_with_anomalies.loc[anomaly_indices, 'y'] += high_value_y
    return data_with_anomalies, anomaly_indices

# Example usage
vehicle_id_to_anomaly = "veh611" # Replace with the desired vehicle ID
anomalous_test_data, test_anomaly_indices_vehicle = inject_constant_speed_offset_vehicle( anomaly_data, vehicle_id=vehicle_id_to_anomaly, high_value_factor=0.6)

# Generate anomaly labels for the test data
anomaly_labels_test = np.zeros(len(anomaly_data), dtype=int)
anomaly_labels_test[test_anomaly_indices_vehicle] = 1

# Display the results
print(f"Injected anomalies for vehicle ID {vehicle_id_to_anomaly}:")
print(f"Labels (0 for normal, 1 for anomalous): {anomaly_labels_test}")
print(f"Shape of labels: {anomaly_labels_test.shape}")


In [ ]:
# Drop the Unneccesary columns as we did for train and test data
anomalous_test_data = anomalous_test_data.drop(columns=['angle', 'id', 'time'])

anomalous_normalized_test = min_max_scaler.transform(anomalous_test_data)
anomalous_normalized_test = pd.DataFrame(anomalous_normalized_test, columns=anomalous_test_data.columns)

anomalous_normalized_test.head(),

**Apply Sliding** to all four data sets and the lables

In [ ]:
import numpy as np


# Define the sliding window function
def sliding_window(data, window_size, stride):
    windows = []
    for i in range(0, len(data) - window_size + 1, stride):
        window = data.iloc[i:i + window_size].values
        windows.append(window)
    return np.array(windows)

# Parameters for the sliding window
window_size = 4  # Number of time steps in each window
stride = 1  # Stride to move the window (1 for maximum overlap)

# Apply the sliding window to the dataset
data_windows = sliding_window(data_normalized_train, window_size, stride)
test_condition_windows = sliding_window(data_normalized_test, window_size, stride)
anomalous_test_windows = sliding_window(anomalous_normalized_test, window_size, stride)
anomalous_normal_windows = sliding_window(data_normalized_anomaly, window_size, stride)

# Reshape to add a channel dimension (samples, time_steps, features, channels)
data_windows_reshaped = data_windows.reshape(data_windows.shape[0],1, data_windows.shape[1], data_windows.shape[2])
anomalous_windows_reshaped = anomalous_test_windows.reshape(anomalous_test_windows.shape[0],1, anomalous_test_windows.shape[1], anomalous_test_windows.shape[2])
test_windows_reshaped = test_condition_windows.reshape(test_condition_windows.shape[0],1, test_condition_windows.shape[1], test_condition_windows.shape[2])
anomalous_normal_reshaped = anomalous_normal_windows.reshape(anomalous_normal_windows.shape[0],1, anomalous_normal_windows.shape[1], anomalous_normal_windows.shape[2])

# Convert to PyTorch tensors
data_windows_tensor = torch.tensor(data_windows_reshaped, dtype=torch.float32)
test_windows_tensor = torch.tensor(test_windows_reshaped, dtype=torch.float32)
anomalous_test_windows_tensor = torch.tensor(anomalous_windows_reshaped, dtype=torch.float32)
anomalous_normal_windows_tensor = torch.tensor(anomalous_normal_reshaped, dtype=torch.float32)

# Print shapes
print("Data Windows Tensor Shape:", data_windows_tensor.shape)
print("Test Condition Windows Tensor Shape:", test_windows_tensor.shape)
print("Anomalous Test Windows Tensor Shape:", anomalous_test_windows_tensor.shape)
print("Anomalous Test Windows Tensor Shape:", anomalous_normal_windows_tensor.shape)

In [ ]:
def sliding_window_labels(labels, window_size, stride):
    windows = []
    for i in range(0, len(labels) - window_size + 1, stride):
        window = labels[i:i + window_size]

        windows.append(int(np.any(window)))
    return np.array(windows)


window_size = 4  # Match your sliding window size
stride =1  # Match your sliding window stride
anomalous_labels_windows = sliding_window_labels(anomaly_labels_test, window_size, stride)
anomalous_labels_windows_tensor = torch.tensor(anomalous_labels_windows, dtype=torch.float32)

print("Windowed Anomalous Labels:", anomalous_labels_windows_tensor)
print(anomalous_labels_windows_tensor.shape)

Load Train and Test dataset

In [ ]:
training_dataloader = DataLoader(data_windows_tensor, batch_size=32, shuffle= False, drop_last=True)
test_dataloader = DataLoader(test_windows_tensor, batch_size=32, shuffle=False, drop_last=True)
print('Length of the training dataloader:', len(training_dataloader))
print('Length of the test dataloader:', len(test_dataloader))

Our VAE architecture

In [ ]:
class Conv_VAE(nn.Module):
    def __init__(self):
        super(Conv_VAE, self).__init__()

        self.encoder = nn.Sequential()
        self.encoder.add_module('conv1', nn.Conv2d(in_channels=1, out_channels=4, kernel_size=2, stride=1))
        self.encoder.add_module('bnorm1', nn.BatchNorm2d(num_features=4))
        #self.encoder.add_module('relu1', nn.ReLU(inplace=True))
        self.encoder.add_module('relu1', nn.LeakyReLU(negative_slope=0.1, inplace=True))
        self.encoder.add_module('conv2', nn.Conv2d(in_channels=4, out_channels=8, kernel_size=2, stride=1))
        self.encoder.add_module('relu2', nn.LeakyReLU(negative_slope=0.1, inplace=True))
        #self.encoder.add_module('relu2', nn.ReLU(inplace=True))
        self.encoder.add_module('conv3', nn.Conv2d(in_channels=8, out_channels=16, kernel_size=2, stride=1))
        self.encoder.add_module('relu3', nn.LeakyReLU(negative_slope=0.1, inplace=True))
        #self.encoder.add_module('relu3', nn.ReLU(inplace=True))

        self._mu = nn.Linear(in_features=16, out_features=4)
        self._logvar = nn.Linear(in_features=16, out_features=4)

        self.decoder = nn.Sequential()
        self.decoder.add_module('tconv3', nn.ConvTranspose2d(in_channels=4, out_channels=8, kernel_size=2, stride=1))
        self.decoder.add_module('relu3', nn.LeakyReLU(negative_slope=0.1, inplace=True))
        #self.encoder.add_module('relu3', nn.ReLU(inplace=True))

        self.decoder.add_module('tconv2', nn.ConvTranspose2d(in_channels=8, out_channels=16, kernel_size=2, stride=1))
        self.decoder.add_module('relu2',nn.LeakyReLU(negative_slope=0.1, inplace=True))
        #self.encoder.add_module('relu2', nn.ReLU(inplace=True))

        self.decoder.add_module('tconv1', nn.ConvTranspose2d(in_channels=16, out_channels=1, kernel_size=2, stride=1))
        self.decoder.add_module('sigmoid1', nn.Sigmoid())

    def reparameterization(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.rand_like(std)
        sampling = mu + (eps * std)
        return sampling

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.shape[0], -1)

        mu = self._mu(x)
        logvar = self._logvar(x)

        x = self.reparameterization(mu, logvar)

        x = x.view(-1, 4, 1, 1)

        return self.decoder(x), mu, logvar

In [ ]:
model = Conv_VAE().to(device)
summary(model, input_size=(32, 1, 4, 4))

In [ ]:
def training_batch(data, model, optimizer):
    model.train()
    #data = data.to(device)
    recon, mu, logvar = model(data)
    loss = loss_function(recon, data, mu, logvar)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
with torch.inference_mode():
    def test_batch(data, model):
        model.eval()
        #data = data.to(device)
        recon, mu, logvar = model(data)
        loss = loss_function(recon, data, mu, logvar)
        return loss

In [ ]:
def loss_function(recon, x, mu, logvar, beta = 1.0):
    RECON = F.mse_loss(recon, x, reduction='sum') / x.size(0)
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return RECON + beta * KLD

In [ ]:
conv_vae = Conv_VAE().to(device)

optimizer = optim.Adam(conv_vae.parameters(), lr=0.001, weight_decay=1e-3)

**Train model for 100 epochs with earlystop to avoid overfitting**

In [ ]:
n_epochs = 100
patience = 20  # Number of epochs to wait for improvement
best_loss = float('inf')  # Initialize best loss as infinity
epochs_without_improvement = 0  # Counter for early stopping

training_loss, test_loss = [], []

for epoch in range(n_epochs):
    training_losses, test_losses = [], []

    # Training loop
    for data in training_dataloader:
        trng_batch_loss = training_batch(data, conv_vae, optimizer)
        training_losses.append(trng_batch_loss.item())
    training_per_epoch_loss = np.array(training_losses).mean()

    # Validation loop
    for data in test_dataloader:
        tst_batch_loss = test_batch(data, conv_vae)
        test_losses.append(tst_batch_loss.item())
    test_per_epoch_loss = np.array(test_losses).mean()

    # Append losses
    training_loss.append(training_per_epoch_loss)
    test_loss.append(test_per_epoch_loss)

    # Check for improvement in validation loss
    if test_per_epoch_loss < best_loss:
        best_loss = test_per_epoch_loss  # Update best loss
        epochs_without_improvement = 0  # Reset counter
    else:
        epochs_without_improvement += 1  # Increment counter

    # Print progress
    if (epoch+1) % 10 == 0:
        print(f'Epoch: {epoch+1}/{n_epochs}\t| Training loss: {training_per_epoch_loss:.4f} |   ', end='')
        print(f'Test loss: {test_per_epoch_loss:.4f}')

    # Early stopping condition
    if epochs_without_improvement >= patience:
        print(f"Early stopping triggered at epoch {epoch+1}. Best test loss: {best_loss:.4f}")
        break


In [ ]:
plt.figure(figsize=(8, 5))

plt.plot(training_loss, 'g-', linewidth=2, label='Training loss')
plt.plot(test_loss, 'c--', linewidth=2, label='Test loss')
plt.title('Loss curve', fontsize=23)
plt.xlabel('No. of epochs', fontsize=17)
plt.ylabel('Losses', fontsize=17)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=14);

**Model Evaluation**

In [ ]:
conv_vae = conv_vae.to(device)  # Move model to the selected device
data_windows_tensor = data_windows_tensor.to(device)
reconstructions, _, _ = conv_vae(data_windows_tensor)

reconstruction_errors = torch.mean((data_windows_tensor - reconstructions) ** 2, dim=[1, 2, 3])


In [ ]:
anomalous_normal_windows_tensor = anomalous_normal_windows_tensor.to(device)
reconstructions_test, _, _ = conv_vae(anomalous_normal_windows_tensor)

reconstruction_errors_test = torch.mean((anomalous_normal_windows_tensor - reconstructions_test) ** 2, dim=[1, 2, 3])  # Per sample error

In [ ]:
anomalous_test_windows_tensor = anomalous_test_windows_tensor.to(device)
reconstructions_anomalous, _, _ = conv_vae(anomalous_test_windows_tensor)

reconstruction_errors_anomalous = torch.mean((anomalous_test_windows_tensor - reconstructions_anomalous) ** 2, dim=[1, 2, 3])  # Per sample error

In [ ]:
from sklearn.metrics import roc_curve, auc

# Convert PyTorch tensors to NumPy arrays
anomalous_labels_windows_numpy = anomalous_labels_windows_tensor.cpu().numpy()
reconstruction_errors_anomalous_numpy = reconstruction_errors_anomalous.detach().cpu().numpy()

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(anomalous_labels_windows_numpy, reconstruction_errors_anomalous_numpy)

# Compute AUC
roc_auc = auc(fpr, tpr)

print("ROC AUC:", roc_auc)
plt.plot(fpr, tpr, label="F1-Score")

In [ ]:
import numpy as np
reconstruction_errors_numpy = reconstruction_errors.detach().cpu().numpy()
# Compute the 95th percentile of reconstruction errors
threshold = np.percentile(reconstruction_errors_numpy, 98)
print("Threshold (80th percentile):", threshold)

In [ ]:
anomaly_predictions = reconstruction_errors_anomalous > threshold
anomaly_predictions_numpy = anomaly_predictions.cpu().numpy()

# Calculate True Positives, False Positives, True Negatives, and False Negatives
tp = ((anomaly_predictions_numpy == 1) & (anomalous_labels_windows_numpy == 1)).sum().item()
fp = ((anomaly_predictions_numpy == 1) & (anomalous_labels_windows_numpy == 0)).sum().item()
tn = ((anomaly_predictions_numpy == 0) & (anomalous_labels_windows_numpy == 0)).sum().item()
fn = ((anomaly_predictions_numpy == 0) & (anomalous_labels_windows_numpy == 1)).sum().item()

# Precision, Recall, and F1 Score
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Accuracy
accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0

# Print results
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}, Accuracy: {accuracy:.4f}")


In [ ]:
import matplotlib.pyplot as plt

reconstruction_errors_test_numpy= reconstruction_errors_test.detach().cpu().numpy()
reconstruction_errors_anomalous_numpy = reconstruction_errors_anomalous.detach().cpu().numpy()
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.hist(reconstruction_errors_test_numpy, bins=50, alpha=0.5)
plt.xlabel("Reconstruction Error Normal")
plt.ylabel("Frequency")
plt.axvline(x=threshold, color='r', linestyle='--', label="Threshold")
plt.subplot(1, 2, 2)
plt.hist(reconstruction_errors_anomalous_numpy, bins=50, alpha=0.5)
plt.axvline(x=threshold, color='r', linestyle='--', label="Threshold")
plt.legend()
#plt.title("Reconstruction Error Distribution")
plt.xlabel("Reconstruction Error Anomalous")
plt.ylabel("Frequency")
plt.show()

**Evaluate our model's real time anoimaly detection performance**

In [ ]:
import time
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from collections import deque
import pandas as pd

# Load your trained VAE model
vae_model = conv_vae  # Replace with your saved model file
vae_model.eval()

# Fit the scaler on training data
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_data1.values)  # Replace with your training dataset

# Define sliding window parameters
window_size = 4  # Number of time steps in each window
stride = 1  # Stride to move the window
sliding_window_buffer = deque(maxlen=window_size)

# Step 1: Simulate Streaming from Test Data
def stream_test_data(test_data, interval=0.1):
    """
    Stream test data row by row, compatible with both DataFrame and NumPy array.
    """
    if isinstance(test_data, pd.DataFrame):
        # DataFrame: Use .iloc to iterate over rows
        for i in range(len(test_data)):
            yield test_data.iloc[i].values  # Yield row as NumPy array
            time.sleep(interval)
    elif isinstance(test_data, np.ndarray):
        # NumPy array: Iterate over rows directly
        for row in test_data:
            yield row  # Yield row
            time.sleep(interval)
    else:
        raise ValueError("Unsupported data format. Must be DataFrame or NumPy array.")

# Step 2: Preprocess Each Row
def preprocess_row(row):
    """
    Normalize a single row after ensuring it's numeric and compatible with the scaler.
    """
    # Ensure the row is a 1D array and reshape it to match the scaler's input shape
    row = np.array(row).reshape(1, -1)  # Reshape to (1, number of features)

    # Ensure the number of features matches the scaler's expectations
    if row.shape[1] != train_data1.shape[1]:
        raise ValueError(f"The input row has {row.shape[1]} features, "
                         f"but {train_data1.shape[1]} features are expected.")

    # Normalize the row using the scaler
    normalized_row = scaler.transform(row)
    return normalized_row[0]  # Return as a 1D array

# Step 3: Real-Time Anomaly Detection with Inference Time Measurement
def real_time_sliding_window_detection_with_inference_time(test_data, threshold=threshold, interval=0.1):
    total_inference_time = 0.0
    num_inferences = 0

    for row in stream_test_data(test_data, interval):
        # Normalize the incoming data row
        normalized_row = preprocess_row(row)

        # Add normalized row to the sliding window buffer
        sliding_window_buffer.append(normalized_row)

        # Process when sliding window is full
        if len(sliding_window_buffer) == window_size:
            # Convert sliding window to NumPy array
            window_array = np.array(sliding_window_buffer)

            # Reshape for CNN input (samples, channels, time_steps, features)
            window_tensor = torch.tensor(window_array.reshape(1, 1, window_array.shape[0], window_array.shape[1]),
                                         dtype=torch.float32)
            #window_tensor = window_tensor.to(device)

            # Measure inference time
            start_time = time.time()

            # Anomaly detection with the VAE
            #reconstructed, mu, logvar = vae_model(window_tensor)
            reconstructed = vae_model(window_tensor)
            reconstruction_error = torch.mean((window_tensor - reconstructed) ** 2).item()

            # Record inference time
            end_time = time.time()
            inference_time = end_time - start_time
            total_inference_time += inference_time
            num_inferences += 1

            # Check if it's an anomaly
            if reconstruction_error > threshold:
                print(f"Anomaly detected! {inference_time:.6f} seconds")

            #else:
               #print(f"Normal data")

            #print(f"Inference Time: {inference_time:.6f} seconds")

    # Calculate average inference time
    average_inference_time = total_inference_time / num_inferences if num_inferences > 0 else 0
    print(f"Average Inference Time: {average_inference_time:.6f} seconds")
    return average_inference_time

# Simulate real-time detection with inference time measurement
average_inference_time = real_time_sliding_window_detection_with_inference_time(
    anomalous_test_data, threshold=threshold, interval=0.1
)
